In [1]:
!wget https://github.com/huseinzol05/Malaya-Dataset/raw/master/english-malay/english-malay1.json

--2019-09-11 09:04:25--  https://github.com/huseinzol05/Malaya-Dataset/raw/master/english-malay/english-malay1.json
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/english-malay/english-malay1.json [following]
--2019-09-11 09:04:26--  https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/english-malay/english-malay1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4379510 (4.2M) [text/plain]
Saving to: ‘english-malay1.json.1’

english-malay1.json 100%[===================>]   4.18M  --.-KB/s    in 0.1s    

2019-09-11 09:04:28 (35.5 MB/s) - ‘english-

In [2]:
import json

with open('english-malay1.json') as fopen:
  malay_english = json.load(fopen)
len(malay_english)

19995

In [3]:
malay_english[:3]

[['Rachel Pike : The science behind a climate headline',
  'rachel pike: sains di sebalik tajuk iklim'],
 ['That report was written by 620 scientists from 40 countries .',
  'laporan itu ditulis oleh 620 saintis dari 40 negara.'],
 ['They wrote almost a thousand pages on the topic .',
  'mereka menulis hampir seribu muka surat mengenai topik itu.']]

In [4]:
!pip3 install malaya
!pip3 install fuzzywuzzy

In [5]:
import malaya
import re

tokenizer = malaya.preprocessing.SocialTokenizer().tokenize

def is_number_regex(s):
    if re.match("^\d+?\.\d+?$", s) is None:
        return s.isdigit()
    return True

def preprocessing(string):
    tokenized = tokenizer(string)
    tokenized = ['<NUM>' if is_number_regex(w) else w for w in tokenized]
    return ' '.join(tokenized)

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
preprocessing('That report was written by 620 scientists from 40 countries .')

'That report was written by <NUM> scientists from <NUM> countries .'

In [0]:
import collections
import json

def build_dataset(words, n_words, atleast=1):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reversed_dictionary

In [0]:
english, malay = [], []
for row in malay_english:
  if len(row[0]) and len(row[1]):
    english.append(row[0])
    malay.append(row[1])

In [9]:
from tqdm import tqdm

for i in tqdm(range(len(english))):
  english[i] = preprocessing(english[i])

100%|██████████| 19979/19979 [00:02<00:00, 7952.65it/s]


In [10]:
for i in tqdm(range(len(malay))):
  malay[i] = preprocessing(malay[i])

100%|██████████| 19979/19979 [00:02<00:00, 8608.54it/s]


In [0]:
concat = ' '.join(english).split()
size_concat = len(set(concat))
english_dictionary, english_reversed_dictionary = build_dataset(concat, size_concat, atleast=1)

In [12]:
len(english_dictionary)

19138

In [0]:
concat = ' '.join(malay).split()
size_concat = len(set(concat))
malay_dictionary, malay_reversed_dictionary = build_dataset(concat, size_concat, atleast=1)

In [14]:
len(malay_dictionary)

13877

In [0]:
GO = malay_dictionary['GO']
PAD = malay_dictionary['PAD']
EOS = malay_dictionary['EOS']
UNK = malay_dictionary['UNK']

In [16]:
english[0]

'Rachel Pike : The science behind a climate headline'

In [17]:
for i in range(len(english)):
    english[i] += ' EOS'
    
english[0]

'Rachel Pike : The science behind a climate headline EOS'

In [0]:
class Translator:
    def __init__(self, size_layer, num_layers, embedded_size,
                 from_dict_size, to_dict_size, learning_rate, batch_size):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,
                                           initializer=tf.orthogonal_initializer(),
                                           reuse=reuse)
          
        def attention(encoder_out, seq_len, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells(reuse) for _ in range(num_layers)]), 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        encoder_embedding = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        encoder_out, encoder_state = tf.nn.dynamic_rnn(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
            inputs = tf.nn.embedding_lookup(encoder_embedding, self.X),
            sequence_length = self.X_seq_len,
            dtype = tf.float32)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        dense = tf.layers.Dense(to_dict_size)
        decoder_cells = attention(encoder_out, self.X_seq_len)
        
        training_helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embedding, decoder_input),
                sequence_length = self.Y_seq_len,
                time_major = False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cells,
                helper = training_helper,
                initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
            
                output_layer = dense)
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
        self.training_logits = training_decoder_output.rnn_output
        
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cells,
                helper = predicting_helper,
                initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                output_layer = dense)
        
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.X_seq_len))
        self.predicting_ids = predicting_decoder_output.sample_id
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
size_layer = 128
num_layers = 2
embedded_size = 128
learning_rate = 1e-3
batch_size = 32
epoch = 10

In [20]:
import tensorflow as tf

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, len(malay_dictionary), 
                len(english_dictionary), learning_rate,batch_size)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [22]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(malay, english)

train_X[0], train_Y[0]

('tetapi selepas itu , perkara - perkara yang cukup mantap dan seragam cantik .',
 'But after that , things were pretty steady and pretty uniform . EOS')

In [0]:
train_X = str_idx(train_X, malay_dictionary)
test_X = str_idx(test_X, malay_dictionary)
train_Y = str_idx(train_Y, english_dictionary)
test_Y = str_idx(test_Y, english_dictionary)

In [24]:
import tqdm
import numpy as np

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        maxlen = max([len(s) for s in train_X[i : index] + train_Y[i : index]])
        batch_x, seq_x = pad_sentence_batch(train_X[i : index], PAD)
        batch_y, seq_y = pad_sentence_batch(train_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
        
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x, seq_x = pad_sentence_batch(test_X[i : index], PAD)
        batch_y, seq_y = pad_sentence_batch(test_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/469 [00:00<?, ?it/s]

epoch 1, training avg loss 5.995502, training avg acc 0.136907
epoch 1, testing avg loss 5.442404, testing avg acc 0.194961


minibatch loop:   0%|          | 0/469 [00:00<?, ?it/s]

epoch 2, training avg loss 5.053892, training avg acc 0.233081
epoch 2, testing avg loss 4.989242, testing avg acc 0.272881


minibatch loop:   0%|          | 0/469 [00:00<?, ?it/s]

...

epoch 8, training avg loss 2.306392, training avg acc 0.574516
epoch 8, testing avg loss 3.567849, testing avg acc 0.511956


minibatch loop:   0%|          | 0/469 [00:00<?, ?it/s]

epoch 9, training avg loss 2.071074, training avg acc 0.605599
epoch 9, testing avg loss 3.636268, testing avg acc 0.524038


minibatch loop: 100%|██████████| 157/157 [00:41<00:00,  4.15it/s, accuracy=0.583, cost=2.7]

epoch 10, training avg loss 1.869579, training avg acc 0.635038
epoch 10, testing avg loss 3.508983, testing avg acc 0.542806
